In [1]:
from plyfile import PlyData, PlyElement
from plotly.graph_objs import *
import math
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:

plydata = PlyData.read('../ply/cow2.ply')

nr_points=plydata.elements[0].count
nr_faces=plydata.elements[1].count

# unpack vertices to array
vertices_pt_xyz =np.array([plydata['vertex'][k] for k in range(nr_points)])
x,y,z=zip(*vertices_pt_xyz)

# unpack faces to array
faces_pt_3v = [plydata['face'][k][0] for k in range(nr_faces)]
faces_pt_3v[0]


points=np.array([plydata['vertex'][k] for k in range(nr_points)])
x,y,z=zip(*points)

faces=[plydata['face'][k][0] for k in range(nr_faces)]
faces[0]

array([0, 1, 2], dtype=int32)

In [3]:
import numpy as np
import matplotlib.cm as cm
from scipy.spatial import Delaunay

def map_z2color(zval, colormap, vmin, vmax):
    #map the normalized value val to a corresponding color in the mpl colormap
    
    if vmin>=vmax:
        raise ValueError('incorrect relation between vmin and vmax')
    t=(zval-vmin)/float((vmax-vmin))#normalize val
    C=map(np.uint8, np.array(colormap(t)[:3])*255)
    #convert to a Plotly color code:
    return 'rgb'+str((C[0], C[1], C[2]))

def mpl_to_plotly(cmap, pl_entries):
    h=1.0/(pl_entries-1)
    pl_colorscale=[]
    for k in range(pl_entries):
        C=map(np.uint8, np.array(cmap(k*h)[:3])*255)
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])
    return pl_colorscale

def plotly_trisurf(x, y, z, simplices, colormap=cm.RdBu, showcolorbar=False, plot_edges=None):
    #x, y, z are lists of coordinates of the triangle vertices 
    #simplices are the simplices that define the triangulation;
    #simplices  is a numpy array of shape (no_triangles, 3)
    #insert here the  type check for input data
    
    points3D=np.vstack((x,y,z)).T
    tri_vertices= points3D[simplices]# vertices of the surface triangles  
    zmean=tri_vertices[:, :, 2].mean(-1)# mean values of z-coordinates of the
                                        #triangle vertices
      
    min_zmean, max_zmean=np.min(zmean), np.max(zmean)
    
    facecolor=[map_z2color(zz,  colormap, min_zmean, max_zmean) for zz in zmean]
    I,J,K=zip(*simplices)
    
    triangles=Mesh3d(x=x,
                     y=y,
                     z=z,
                     facecolor=facecolor, 
                     i=I,
                     j=J,
                     k=K,
                     name=''
                    )
    
    if showcolorbar==True:
        pl_colorsc=mpl_to_plotly(colormap,11)
        # define a fake Scatter3d trace in order to enable displaying the colorbar for the trisurf
        
        colorbar=Scatter3d(x=x[:2],
                           y=y[:2],
                           z=z[:2],
                           mode='markers',
                           marker=dict(size=0.1, color=[min_zmean, max_zmean], 
                                      colorscale=pl_colorsc, showscale=True),
                             hoverinfo='None'
                          )
    
    
    if plot_edges is None: # the triangle sides are not plotted 
        if  showcolorbar is True:
            return Data([colorbar, triangles])
        else: 
            return  Data([triangles])
    else:#plot edges
        #define the lists Xe, Ye, Ze, of x, y, resp z coordinates of edge end points for each triangle
        #None separates data corresponding to two consecutive triangles
        Xe=[]
        Ye=[]
        Ze=[]
        for T in tri_vertices:
            Xe+=[T[k%3][0] for k in range(4)]+[ None]
            Ye+=[T[k%3][1] for k in range(4)]+[ None]
            Ze+=[T[k%3][2] for k in range(4)]+[ None]
       
        #define the lines to be plotted
        lines=Scatter3d(x=Xe,
                        y=Ye,
                        z=Ze,
                        mode='lines',
                        line=Line(color= 'rgb(50,50,50)', width=1.5)
               )
        if  showcolorbar is True:
            return Data([colorbar, triangles, lines])
        else: 
            
            return Data([triangles, lines])

In [4]:
#x,y,z=zip(*vertices_pt_xyz)

# points=np.array([plydata['vertex'][k] for k in range(nr_points)])
# x,y,z=zip(*points)

# faces=[plydata['face'][k][0] for k in range(nr_faces)]
# faces[0]

cowSurf = plotly_trisurf(x,y,z, faces_pt_3v, colormap=cm.RdBu, plot_edges=None)


xMark =Scatter3d(x=[5],
            y=[0],
            z=[0],
            mode='markers',
            marker=dict(size=15,color='#f44242')
           )

yMark =Scatter3d(x=[0],
            y=[5],
            z=[0],
            mode='markers',
            marker=dict(size=15,color='#50ed7a')
           )

zMark =Scatter3d(x=[0],
            y=[0],
            z=[-5],
            mode='markers',
            marker=dict(size=15,color='#3f52e2')
           )

        
axis=dict(showbackground=False,
          showline=True,  
          zeroline=True,
          showgrid=True,
          showticklabels=True,
          title='' 
          )

layout = Layout(title="PLY", 
                width=800,
                height=800,
                showlegend=False,
                scene=Scene(aspectmode='manual',
                            aspectratio=dict(x=1, y=1, z=1),
                            xaxis=dict(range=[-10, 10], title='x Axis'),
                            yaxis=dict(range=[-10, 10], title='y Axis'),
                            zaxis=dict(range=[-20,0], title='z Axis'),
                            camera = dict(
                                up=dict(x=1, y=0, z=0),
                                center=dict(x=0, y=0, z=0),
                                eye=dict(x=0, y=0, z=2)
                            ),

                           ),
                margin=Margin(t=100),
                hovermode='closest',
                
                )


data=Data([cowSurf,xMark,yMark,zMark])

noaxis=dict(showbackground=False,
            showline=False,  
            zeroline=False,
            showgrid=False,
            showticklabels=False,
            title='' 
          )

fig3 = Figure(data=cowSurf)

iplot(fig3)

PlotlyListEntryError: Invalid entry found in 'data' at index, '0'

Path To Error: [0]

Valid items for 'data' at path [] under parents []:
    ['Area', 'Bar', 'Box', 'Candlestick', 'Carpet', 'Choropleth',
    'Contour', 'Contourcarpet', 'Heatmap', 'Heatmapgl', 'Histogram',
    'Histogram2d', 'Histogram2dcontour', 'Mesh3d', 'Ohlc', 'Parcoords',
    'Pie', 'Pointcloud', 'Scatter', 'Scatter3d', 'Scattercarpet',
    'Scattergeo', 'Scattergl', 'Scattermapbox', 'Scatterternary',
    'Surface']

Entry should subclass dict.